In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy

# Poisson Equations

<font  face=Times color=darkblue size=3> In this notebook, we are gonna show some computational methods to firstly solve Poisson equation: $$\Delta \phi (\vec x_{\vec r})=-\frac{1}{\epsilon_0}\rho(\vec x_{\vec r}).$$ After discretization through the centered three-point formula in each direction, it has a form: $$\phi (\vec x_{\vec r})=\frac{1}{2d}\sum_1^d[\phi (\vec x_{\vec r+h\vec e_i})+\phi (\vec x_{\vec r-h\vec e_i})]+\frac{h^2}{2d\epsilon_0}\rho(\vec x_{\vec r}).$$
    For simplicity, we set the time spacing $h=1$, $\epsilon_0=1$ and we consider $N\times N$ two-dimensional cases($d=2$ and including the boundary) with the initial condition and charge density defined as below. We use Dirichlet boundary conditions.

In [2]:
### you can change the initial conditions in this cell for all methods below, including N and the charge_density

N=10 #2 of N are boundaries
d=2
epsilon=1
h=1
#initial condition
potential_initial=np.zeros((N,N)) #this includes boundary=0
charge_density=np.ones((N,N))   #uniform distribution

## Jacobi Relaxation

<font  face=Times color=darkblue size=3> We use the discretized formula to iteratively compute new $\phi$ and replace the old values by new ones until $\delta \phi={\rm max}_{\vec r}|\phi^{\rm new}(\vec x_{\vec r})-\phi(\vec x_{\vec r})|$ is no larger than the specified bound which is defined below.

In [3]:
def jacobi_relaxation(bound_jr):
    iterations=0
    potential_here=copy.deepcopy(potential_initial)
    while True:
        iterations+=1
        potential_new=copy.deepcopy(potential_here)  #set phi_new
        for i in range(1,N-1):
            for j in range(1,N-1): # simulate all N-2*N-2 sites using discretized poisson equation, no simulation for boundary 
                left_index=i-h
                right_index=i+h
                up_index=j+h
                down_index=j-h
                potential_new[i,j]=(1/(2*d))*(potential_new[i,up_index]+potential_new[i,down_index]+potential_new[right_index,j]
                                              +potential_new[left_index,j])+(h**2/(2*d*epsilon))*charge_density[i,j] 
        max_delta=np.max(np.abs(potential_new-potential_here)) # compute delta_phi
        if max_delta <= bound_jr:
            break
        else:
            potential_here=copy.deepcopy(potential_new)
        #print(iterations,max_delta)
    return iterations,potential_new

In [4]:
iterations_jr,potential_jr=jacobi_relaxation(0.0001)

In [5]:
iterations_jr #iterations_needed

74

In [6]:
potential_jr

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.21352645, 1.92708216, 2.33545848, 2.52300868,
        2.52302669, 2.33550608, 1.92714119, 1.2135706 , 0.        ],
       [0.        , 1.92708216, 3.15943242, 3.89184808, 4.23365662,
        4.23368844, 3.89193213, 3.15953666, 1.92716012, 0.        ],
       [0.        , 2.33545848, 3.89184808, 4.83899752, 5.28623545,
        5.28627573, 4.83910393, 3.89198006, 2.33555718, 0.        ],
       [0.        , 2.52300868, 4.23365662, 5.28623545, 5.78618964,
        5.78623268, 5.28634916, 4.23379765, 2.52311414, 0.        ],
       [0.        , 2.52302669, 4.23368844, 5.28627573, 5.78623268,
        5.78627313, 5.28638259, 4.23382096, 2.5231258 , 0.        ],
       [0.        , 2.33550608, 3.89193213, 4.83910393, 5.28634916,
        5.28638259, 4.83919223, 3.89204165, 2.33558797, 0.        ],
       [0.        , 1.92714119, 3.1595366

## Gauss-Seidel Relaxation

<font  face=Times color=darkblue size=3> In Jacobi Relaxation, we need to store two arrays of potential, including the new and old ones. A simple modification is made here, that is, we update every lattice point which results a lattice with both old and new points.

In [7]:
def gauss_seidel_relaxation(bound_here):
    iterations=0
    potential_new=copy.deepcopy(potential_initial)
    while True:
        iterations+=1
        max_delta=0
        for i in range(1,N-1):
            for j in range(1,N-1): # simulate all N-2*N-2 sites using discretized poisson equation, no simulation for boundary 
                left_index=i-h
                right_index=i+h
                up_index=j+h
                down_index=j-h
                potential_point_new=(1/(2*d))*(potential_new[i,up_index]+potential_new[i,down_index]+potential_new[right_index,j]
                                              +potential_new[left_index,j])+(h**2/(2*d*epsilon))*charge_density[i,j] #new point
                max_delta=max(max_delta,np.abs(potential_new[i,j]-potential_point_new))  #delta_phi
                potential_new[i,j]=potential_point_new #update point
        if max_delta <= bound_here:
            break
        #print(iterations,max_delta)
    return iterations,potential_new

In [8]:
iterations_gs,potential_gs=gauss_seidel_relaxation(0.0001)

In [9]:
iterations_gs

74

In [10]:
potential_gs

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.21352645, 1.92708216, 2.33545848, 2.52300868,
        2.52302669, 2.33550608, 1.92714119, 1.2135706 , 0.        ],
       [0.        , 1.92708216, 3.15943242, 3.89184808, 4.23365662,
        4.23368844, 3.89193213, 3.15953666, 1.92716012, 0.        ],
       [0.        , 2.33545848, 3.89184808, 4.83899752, 5.28623545,
        5.28627573, 4.83910393, 3.89198006, 2.33555718, 0.        ],
       [0.        , 2.52300868, 4.23365662, 5.28623545, 5.78618964,
        5.78623268, 5.28634916, 4.23379765, 2.52311414, 0.        ],
       [0.        , 2.52302669, 4.23368844, 5.28627573, 5.78623268,
        5.78627313, 5.28638259, 4.23382096, 2.5231258 , 0.        ],
       [0.        , 2.33550608, 3.89193213, 4.83910393, 5.28634916,
        5.28638259, 4.83919223, 3.89204165, 2.33558797, 0.        ],
       [0.        , 1.92714119, 3.1595366

## Successive Overrelaxation (SOR)

<font  face=Times color=darkblue size=3> To further fasten the Gauss-Seidel relaxation, we modify the update step as follows: $$\phi (\vec x_{\vec r})\rightarrow (1-w)\phi (\vec x_{\vec r})+w[ \frac{1}{2d}\sum_1^d[\phi (\vec x_{\vec r+h\vec e_i})+\phi (\vec x_{\vec r-h\vec e_i})]+\frac{h^2}{2d\epsilon_0}\rho(\vec x_{\vec r})].$$ A relaxation parameter $w$ is introduced, and the Gauss Seidel corresponds to $w=1$. We now try to use an overrelaxation parameter $w>1$ and this method is only stable for $w<2$. In particular, there is an optimal value $w_{\rm opt}$. In the case of a $N_x \times N_y$ ($N_x=N_y=N-2$) lattice with Dirichlet boundary conditions, $$w_{\rm opt}=\frac{2}{1+\sqrt{1-r^2}},\ r=0.5(\cos{(\pi/N_x)}+\cos{(\pi/N_y)}).$$

In [11]:
def sor(bound_here):
    iterations=0
    potential_new=copy.deepcopy(potential_initial)
    rhere=np.cos(np.pi/(N-2))
    w_opt=2/(1+np.sqrt(1-rhere**2))
    print(w_opt)
    while True:
        iterations+=1
        max_delta=0
        for i in range(1,N-1):
            for j in range(1,N-1): # simulate all N-2*N-2 sites using discretized poisson equation, no simulation for boundary 
                left_index=i-h
                right_index=i+h
                up_index=j+h
                down_index=j-h
                potential_point_new=(1-w_opt)*potential_new[i,j]+w_opt*((1/(2*d))*(potential_new[i,up_index]
                                                                                   +potential_new[i,down_index]
                                                                                   +potential_new[right_index,j]
                                              +potential_new[left_index,j])+(h**2/(2*d*epsilon))*charge_density[i,j]) #new point
                max_delta=max(max_delta,np.abs(potential_new[i,j]-potential_point_new))  #delta_phi
                potential_new[i,j]=potential_point_new #update point
        if max_delta <= bound_here:
            break
        #print(iterations,max_delta)
    return iterations,potential_new

In [12]:
iterations_sor,potential_sor=sor(0.0001)

1.4464626921716894


In [13]:
iterations_sor

27

In [14]:
potential_sor

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.2136007 , 1.92722626, 2.33565498, 2.5232315 ,
        2.52324641, 2.33569493, 1.92727726, 1.21364049, 0.        ],
       [0.        , 1.92722626, 3.15970651, 3.89221634, 4.23406954,
        4.23409204, 3.89227668, 3.15978354, 1.92728637, 0.        ],
       [0.        , 2.33565498, 3.89221634, 4.83948675, 5.28677923,
        5.2868036 , 4.83955207, 3.89229974, 2.33572006, 0.        ],
       [0.        , 2.5232315 , 4.23406954, 5.28677923, 5.78678991,
        5.78681219, 5.28683894, 4.23414576, 2.52329098, 0.        ],
       [0.        , 2.52324641, 4.23409204, 5.2868036 , 5.78681219,
        5.78683009, 5.2868516 , 4.23415331, 2.52329421, 0.        ],
       [0.        , 2.33569493, 3.89227668, 4.83955207, 5.28683894,
        5.2868516 , 4.839586  , 3.89231998, 2.33572872, 0.        ],
       [0.        , 1.92727726, 3.1597835

<font  face=Times color=darkblue size=3> With the same bound, we can see faster convergence.

## Matrix-Formulation

<font  face=Times color=darkblue size=3> The discretization of Poisson equation leads to aa set of linear algebraic equations, which can be expressed as $$A\vec \phi = \vec b,$$ 
    where,$$\vec\phi=\left(
    \begin{array} {c}
    \phi(\vec x_{r_0})\\
    \phi(\vec x_{r_1})\\
    \vdots\\
    \phi(\vec x_{r_{N_{\rm site}-1}})
    \end{array}
    \right),
    $$ ${\rm N_{site}}=(N-2)^2$, $\vec b$ is set by the boundary conditions($b_i=\frac{h^2}{\epsilon_0}\rho_i+\sum\phi_{\rm nearest\ boundary}$) and the charge distribution, and $A$ is given by distribution of Laplace operator ($A(i,i)=2*d$ and $A(i,j)=1$ only if the points are neighbors).
    <br><br> Below we build the matrices.

In [15]:
### written for 2d

N_site=(N-2)**2
A_mat=np.zeros((N_site,N_site))
for i in range(N-2):
    for j in range(N-2):
        neighbor_list_row=[]
        if j==0:
            neighbor_list_row.append(j+1)
        elif j==N-3:
            neighbor_list_row.append(j-1)
        else:
            neighbor_list_row.append(j-1)
            neighbor_list_row.append(j+1)
        neighbor_list_col=[]
        if i==0:
            neighbor_list_col.append(i+1)
        elif i==N-3:
            neighbor_list_col.append(i-1)
        else:
            neighbor_list_col.append(i-1)
            neighbor_list_col.append(i+1)
        for it in neighbor_list_row:
            A_mat[i*(N-2)+j,i*(N-2)+it]=-1
        for it in neighbor_list_col:
            A_mat[i*(N-2)+j,it*(N-2)+j]=-1
        A_mat[i*(N-2)+j,i*(N-2)+j]=2*d

In [16]:
A_mat

array([[ 4., -1.,  0., ...,  0.,  0.,  0.],
       [-1.,  4., -1., ...,  0.,  0.,  0.],
       [ 0., -1.,  4., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  4., -1.,  0.],
       [ 0.,  0.,  0., ..., -1.,  4., -1.],
       [ 0.,  0.,  0., ...,  0., -1.,  4.]])

In [17]:
b_mat=np.zeros(N_site)   # each site would be the same as we use uniform charge distribution and the boundary is 0
for i in range(N-2):
    for j in range(N-2):
        value_here=charge_density[i+1,j+1]*(h**2)/epsilon
        if i==0:
            value_here+=potential_initial[i+1-1,j+1]
        if i==N-3:
            value_here+=potential_initial[i+1+1,j+1]
        if j==0:
            value_here+=potential_initial[i+1,j+1-1]
        if j==N-3:
            value_here+=potential_initial[i+1,j+1+1]
        b_mat[i*(N-2)+j]=value_here

In [18]:
b_mat

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

## Gauss Elimination

<font  face=Times color=darkblue size=3> We first use the standard procejure to solve the equations which you should be familiar with: Firstly use forward elimination to make $A$ an upper triangular matrix $\tilde A$ with $\vec{\tilde b}$, and then do back substitution to obtain $\phi$.

In [19]:
potential_gauss=np.zeros(N_site) 
#foward 
b_mat_gauss=copy.deepcopy(b_mat)
A_mat_gauss=copy.deepcopy(A_mat)
for j in range(N_site-1):
    for i in range(j+1,N_site):
        f_here=-A_mat_gauss[i,j]/A_mat_gauss[j,j]
        for ind_k in range(j,N_site):
            A_mat_gauss[i,ind_k]=A_mat_gauss[i,ind_k]+f_here*A_mat_gauss[j,ind_k]
        b_mat_gauss[i]=b_mat_gauss[i]+f_here*b_mat_gauss[j]

#back
potential_gauss[-1]=b_mat_gauss[-1]/A_mat_gauss[-1,-1]
for i in reversed(range(N_site-1)):
    sum_here=0
    for j in range(i+1,N_site):
        sum_here+=A_mat_gauss[i,j]*potential_gauss[j]
    potential_gauss[i]=(b_mat_gauss[i]-sum_here)/A_mat_gauss[i,i]

In [20]:
potential_gauss #result

array([1.2136515 , 1.927303  , 2.33573807, 2.52330744, 2.52330744,
       2.33573807, 1.927303  , 1.2136515 , 1.927303  , 3.15982242,
       3.89234184, 4.23418424, 4.23418424, 3.89234184, 3.15982242,
       1.927303  , 2.33573807, 3.89234184, 4.83962264, 5.28690344,
       5.28690344, 4.83962264, 3.89234184, 2.33573807, 2.52330744,
       4.23418424, 5.28690344, 5.78690344, 5.78690344, 5.28690344,
       4.23418424, 2.52330744, 2.52330744, 4.23418424, 5.28690344,
       5.78690344, 5.78690344, 5.28690344, 4.23418424, 2.52330744,
       2.33573807, 3.89234184, 4.83962264, 5.28690344, 5.28690344,
       4.83962264, 3.89234184, 2.33573807, 1.927303  , 3.15982242,
       3.89234184, 4.23418424, 4.23418424, 3.89234184, 3.15982242,
       1.927303  , 1.2136515 , 1.927303  , 2.33573807, 2.52330744,
       2.52330744, 2.33573807, 1.927303  , 1.2136515 ])

In [21]:
# use inverse matrix
np.dot(np.linalg.inv(A_mat),b_mat.reshape(N_site,1))

array([[1.2136515 ],
       [1.927303  ],
       [2.33573807],
       [2.52330744],
       [2.52330744],
       [2.33573807],
       [1.927303  ],
       [1.2136515 ],
       [1.927303  ],
       [3.15982242],
       [3.89234184],
       [4.23418424],
       [4.23418424],
       [3.89234184],
       [3.15982242],
       [1.927303  ],
       [2.33573807],
       [3.89234184],
       [4.83962264],
       [5.28690344],
       [5.28690344],
       [4.83962264],
       [3.89234184],
       [2.33573807],
       [2.52330744],
       [4.23418424],
       [5.28690344],
       [5.78690344],
       [5.78690344],
       [5.28690344],
       [4.23418424],
       [2.52330744],
       [2.52330744],
       [4.23418424],
       [5.28690344],
       [5.78690344],
       [5.78690344],
       [5.28690344],
       [4.23418424],
       [2.52330744],
       [2.33573807],
       [3.89234184],
       [4.83962264],
       [5.28690344],
       [5.28690344],
       [4.83962264],
       [3.89234184],
       [2.335

<font  face=Times color=darkblue size=3> Now we have the solution the same as that if we directly use ${\rm inv}(A)\cdot b$ (for quick confirmation), and the deviations of iterative solutions from this are less than the bound we set.

## $LU$ Decomposition

<font  face=Times color=darkblue size=3>Performing the Gauss elimination, we obtain a triangular matrix $$\tilde A=MA,\ {\rm where} M=M^{(N_{\rm site}-2)}\cdots M^{(1)}M^{(0)}$$ with Frobenius matrices $M^{(j)}$. (i) the diagonal matrix elements $=1$, and (ii) the matrix elements below the diagonal in the $j$-th column are equal to the values of $f$s used in the $j$-th Gauss elimination step. Therefore, $$A=M^{-1}\cdot\tilde A=L\cdot U$$ is the product of the lower triangular matrix $L$ and upper triangular matrix $U$.
    <br><br>
    Since $A\vec\phi=LU\vec\phi=L(U\vec\phi)=\vec b$, we first solve triangular system $L\vec y=\vec b$ using forward substitution, obtaining $\vec y$, and then solve $U\vec \phi=\vec y$ using back substitution. 
    <br><br>
    A clever algorithm by Crout to calculate $LU$ instead of Gauss elimination: Crout fixes $L_{ii}=1$ and then calculate the other elements successively as :
    <br><br> For $j=0,1,\dots,n-1$ do: $$U_{ij}=A_{ij}\sum_{k=0}^{i-1}L_{ik}U_{kj},\ \ i=0,\dots,j$$
    $$L_{ij}=\frac{1}{U_{jj}}[A_{ij}-\sum_{k=0}^{j-1}L_{ik}U_{kj}],\ \ i=j+1,\dots,N_{\rm site}-1$$

In [22]:
b_mat_lu=copy.deepcopy(b_mat)   #original b
l_mat_lu=np.zeros((N_site,N_site)) # to be decided
u_mat_lu=np.zeros((N_site,N_site)) #directly use the matrix we got in Gauss elimination
y_mat_lu=np.zeros(N_site)
A_mat_lu=copy.deepcopy(A_mat) #original A


In [23]:
l_mat_lu=np.eye(N_site)
for j in range(N_site):
    for i in range(0,j+1):
        sum_u=0
        if i>0:
            for indk in range(0,i):
                sum_u+=l_mat_lu[i,indk]*u_mat_lu[indk,j]
        u_mat_lu[i,j]=A_mat_lu[i,j]-sum_u
    for i in range(j+1,N_site):
        sum_l=0
        if j>0:
            for indk in range(0,j):
                sum_l+=l_mat_lu[i,indk]*u_mat_lu[indk,j]
        l_mat_lu[i,j]=(A_mat_lu[i,j]-sum_l)/u_mat_lu[j,j]

In [24]:
y_mat_lu[0]=b_mat_lu[0]/l_mat_lu[0,0]
for i in range(1,N_site):
    sum_here=0
    for j in range(i):
        sum_here+=l_mat_lu[i,j]*y_mat_lu[j]
    y_mat_lu[i]=(b_mat_lu[i]-sum_here)/l_mat_lu[i,i]

In [25]:
potential_lu=np.zeros(N_site)
potential_lu[-1]=y_mat_lu[-1]/u_mat_lu[-1,-1]
for i in reversed(range(0,N_site-1)):
    sum_here=0
    for j in range(i+1,N_site):
        sum_here+=u_mat_lu[i,j]*potential_lu[j]
    potential_lu[i]=(y_mat_lu[i]-sum_here)/u_mat_lu[i,i]

In [26]:
potential_lu # result

array([1.2136515 , 1.927303  , 2.33573807, 2.52330744, 2.52330744,
       2.33573807, 1.927303  , 1.2136515 , 1.927303  , 3.15982242,
       3.89234184, 4.23418424, 4.23418424, 3.89234184, 3.15982242,
       1.927303  , 2.33573807, 3.89234184, 4.83962264, 5.28690344,
       5.28690344, 4.83962264, 3.89234184, 2.33573807, 2.52330744,
       4.23418424, 5.28690344, 5.78690344, 5.78690344, 5.28690344,
       4.23418424, 2.52330744, 2.52330744, 4.23418424, 5.28690344,
       5.78690344, 5.78690344, 5.28690344, 4.23418424, 2.52330744,
       2.33573807, 3.89234184, 4.83962264, 5.28690344, 5.28690344,
       4.83962264, 3.89234184, 2.33573807, 1.927303  , 3.15982242,
       3.89234184, 4.23418424, 4.23418424, 3.89234184, 3.15982242,
       1.927303  , 1.2136515 , 1.927303  , 2.33573807, 2.52330744,
       2.52330744, 2.33573807, 1.927303  , 1.2136515 ])

<font  face=Times color=darkblue size=3> Again, we get the same result.

## Conjugate Gradient Method

<font  face=Times color=darkblue size=3> For a symmetric positive-definite $n\times n$ matrix $A$, and a set of equations $A\vec x=\vec b$, we can define
    $$f(\vec x)=\frac{1}{2}\vec x\cdot A\vec x-\vec b\vec x,$$
    so that the minimum of $f$ is a solution to the system of equations. One way to find the minimum is the method of steepest descent with an arbitrary $\vec x_0$. Then the next point is $\vec x_1=\vec x_0+\lambda_0\vec r_0,$ with $\vec r_0=-\nabla f(\vec x_0)=-A\vec x_0+\vec b$, and with $\frac{d}{d\lambda_0}f(\vec x_0+\lambda_0\vec r_0)=0\ \Rightarrow\ \lambda_0=\frac{\vec r_0\cdot\vec r_0}{\vec r_0\cdot A\vec r_0}.$ Continue the iteration until $|\vec r_k|<\epsilon$.
   <br> 
    To minimize the steps, there is a way to choose the direction at $\vec x_1$ such that the gradient keeps vanishing along the previous direction $\vec r_0$. This implies not only $\vec r_0\perp\nabla f(\vec x_1)$, but also $\vec r_0\perp\nabla f(\vec x_1+\lambda_1\vec r_1)\ \Rightarrow\ \vec r_0\cdot A\vec r_1=0$.
    Such vectors $\vec r_0$ and $\vec r_1$ are called A-conjugare vectors.
    <br><br>
    To have explicit construction by Hestens and Stiefel:
    $$\vec r_0=\vec g_0=-\nabla f(\vec x_0)$$
    $$\vec x_{k+1}=\vec x_k+\lambda_k\vec r_k,\ \ \lambda_k=\frac{\vec g_k\cdot\vec g_k}{\vec r_k\cdot A\vec r_k}$$
    $$\vec g_{k+1}=\vec g_k-\lambda_k\cdot A\vec r_k$$
    $$\vec r_{k+1}=\vec g_{k+1}+\frac{\vec g_{k+1}\cdot\vec g_{k+1}}{\vec g_k\cdot\vec g_k}\vec r_k.$$

In [27]:
A_mat_cg=copy.deepcopy(A_mat)
b_mat_cg=copy.deepcopy(b_mat)
potential_cg=np.zeros(N_site)

In [28]:
r_cg=g_cg=-np.dot(A_mat_cg,potential_cg.reshape(N_site,1))+b_mat_cg.reshape(N_site,1)

In [29]:
bound_cg=0.0001
iter_cg=0
while True:
    iter_cg+=1
    lambda_cg=np.dot(g_cg.reshape(1,N_site),g_cg)/np.dot(r_cg.reshape(1,N_site),np.dot(A_mat_cg,r_cg))
    potential_cg=potential_cg.reshape(N_site,1)+lambda_cg*r_cg
    g_cg_new=g_cg-lambda_cg*np.dot(A_mat_cg,r_cg)
    r_cg=g_cg_new+(np.dot(g_cg_new.reshape(1,N_site),g_cg_new)/np.dot(g_cg.reshape(1,N_site),g_cg))*r_cg
    if np.linalg.norm(r_cg)<bound_cg:
        break
    else:
        g_cg=copy.deepcopy(g_cg_new)

In [30]:
potential_cg

array([[1.2136515 ],
       [1.927303  ],
       [2.33573807],
       [2.52330744],
       [2.52330744],
       [2.33573807],
       [1.927303  ],
       [1.2136515 ],
       [1.927303  ],
       [3.15982242],
       [3.89234184],
       [4.23418424],
       [4.23418424],
       [3.89234184],
       [3.15982242],
       [1.927303  ],
       [2.33573807],
       [3.89234184],
       [4.83962264],
       [5.28690344],
       [5.28690344],
       [4.83962264],
       [3.89234184],
       [2.33573807],
       [2.52330744],
       [4.23418424],
       [5.28690344],
       [5.78690344],
       [5.78690344],
       [5.28690344],
       [4.23418424],
       [2.52330744],
       [2.52330744],
       [4.23418424],
       [5.28690344],
       [5.78690344],
       [5.78690344],
       [5.28690344],
       [4.23418424],
       [2.52330744],
       [2.33573807],
       [3.89234184],
       [4.83962264],
       [5.28690344],
       [5.28690344],
       [4.83962264],
       [3.89234184],
       [2.335

In [31]:
iter_cg

10